In [1]:
!pip install -q elasticsearch ragas datasets langchain-openai


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import json
from getpass import getpass
from elasticsearch import Elasticsearch
from ragas import evaluate
from ragas.metrics import faithfulness, context_recall, context_precision
from datasets import Dataset
from langchain_openai import ChatOpenAI

In [3]:
es = Elasticsearch(
    getpass("Host: "),
    api_key=getpass("API Key: "),
)

Host:  ········
API Key:  ········


In [4]:
index_name = "ragas-books"

In [5]:
def embed_query(text: str):
    res = es.ml.infer_trained_model(
        model_id=".multilingual-e5-small_linux-x86_64",
        body={"docs": [{"text_field": f"query: {text}"}]}
    )
    
    vec = res["inference_results"][0]["predicted_value"]
    
    return vec

In [6]:
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"Deleted existing index '{index_name}'")

es.indices.create(index=index_name, body={
    "mappings": {
        "properties": {
            "book_title": {"type": "text"},
            "author_name": {"type": "text"},
            "book_description": {"type": "text"},
            "rating_score": {"type": "float"},
            "embedding": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
})
print(f"Created index '{index_name}'")

Deleted existing index 'ragas-books'
Created index 'ragas-books'


In [7]:
with open("books.json") as f:
    books = json.load(f)

for i, book in enumerate(books, 1):
    try:
        book["embedding"] = embed_query(book["book_description"])
        es.index(index=index_name, document=book)
        print(f"Indexed {i}: {book['book_title']}")
    except Exception as e:
        print(f"Failed to index '{book.get('book_title', 'Unknown')}': {e}")

Indexed 1: Lucky 7
Indexed 2: Salvation Lost
Indexed 3: Alien Warrior's Mate
Indexed 4: On the Steel Breeze
Indexed 5: Salvage Marines
Indexed 6: Trade Secret
Indexed 7: There Will Be Time
Indexed 8: Only in Death
Indexed 9: His To Claim
Indexed 10: Savage Drift
Indexed 11: Light of the Jedi
Indexed 12: Mega Robo Bros
Indexed 13: Transmetropolitan, Vol. 1: Back on the Street
Indexed 14: The Queen of Traitors
Indexed 15: The Island of Doctor Moreau
Indexed 16: Human Nature
Indexed 17: Legion
Indexed 18: Wolfsbane
Indexed 19: Lamb: The Gospel According to Biff, Christ's Childhood Pal
Indexed 20: Our Pet
Indexed 21: The Aylesford Skull
Indexed 22: Ghosts of War
Indexed 23: The Book of Time
Indexed 24: Because It Is My Blood
Indexed 25: The Annals of the Heechee


In [8]:
def vector_search(query, top_k=3):
    query_vector = embed_query(query)
    
    body = {
        "knn": {
            "field": "embedding",
            "k": top_k,
            "num_candidates": 100,
            "query_vector": query_vector
        },
        "_source": ["book_title", "author_name", "book_description", "rating_score"]
    }
    
    res = es.search(index=index_name, body=body)
    hits = res["hits"]["hits"]
    contexts, books_info = [], []
    
    for hit in hits:
        book = hit["_source"]
        context = f"{book['book_title']} by {book['author_name']}: {book['book_description']}"
        contexts.append(context)
        books_info.append(book)
    
    return contexts, books_info

In [9]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("OPENAI_API_KEY: ")

API_KEY = os.environ["OPENAI_API_KEY"]

chat_llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.1,
    api_key=API_KEY
)

In [10]:
def generate_answer(question, contexts):
    context_text = "\n\n".join(contexts)
    
    prompt = f"""You are a helpful assistant that recommends books.
Use only the information from the context below to answer the question.
Do not include any books, authors, or details that are not explicitly present in the context.

Repeat the exact book title and author from the context in your answer.

Context:
{context_text}

Question:
{question}

Answer:"""
    response = chat_llm.invoke(prompt)
    
    return response.content.strip()

In [11]:
def analyze_question_intent(question):
    question_lower = question.lower()
    
    intent_patterns = {
        'genre_specific': {
            'science fiction': ['science fiction', 'sci-fi', 'space', 'future', 'alien', 'technology'],
            'fantasy': ['fantasy', 'magic', 'dragon', 'wizard', 'medieval', 'kingdom'],
            'mystery': ['mystery', 'detective', 'crime', 'murder', 'investigation', 'thriller'],
            'romance': ['romance', 'love', 'relationship', 'romantic'],
            'horror': ['horror', 'scary', 'ghost', 'supernatural', 'fear'],
            'historical': ['historical', 'history', 'war', 'period', 'ancient'],
            'biography': ['biography', 'memoir', 'life story', 'autobiography'],
            'non-fiction': ['non-fiction', 'nonfiction', 'factual', 'real', 'educational']
        },
        'quality_indicators': {
            'high_rating': ['high rating', 'highly rated', 'best rated', 'top rated', 'excellent'],
            'popular': ['popular', 'bestseller', 'well-known', 'famous', 'acclaimed'],
            'award_winning': ['award', 'prize', 'winner', 'acclaimed', 'celebrated'],
            'classic': ['classic', 'timeless', 'masterpiece', 'legendary'],
            'recent': ['recent', 'new', 'latest', 'modern', 'contemporary']
        },
        'author_focus': ['author', 'writer', 'by', 'written by']
    }
    
    detected_genres = []
    for genre, keywords in intent_patterns['genre_specific'].items():
        if any(keyword in question_lower for keyword in keywords):
            detected_genres.append(genre)
    
    quality_preferences = []
    for quality_type, keywords in intent_patterns['quality_indicators'].items():
        if any(keyword in question_lower for keyword in keywords):
            quality_preferences.append(quality_type)
    
    author_focused = any(keyword in question_lower for keyword in intent_patterns['author_focus'])
    
    return {
        'genres': detected_genres,
        'quality_preferences': quality_preferences,
        'author_focused': author_focused,
        'question_lower': question_lower,
        'genre_keywords': intent_patterns['genre_specific']
    }

In [12]:
def calculate_book_score(book, intent_data):
    score = 0
    reasons = []
    
    rating = float(book.get('rating_score', 0))
    score += rating * 10
    
    book_title = book.get('book_title', '').lower()
    book_desc = book.get('book_description', '').lower()
    author_name = book.get('author_name', '')
    
    for genre in intent_data['genres']:
        genre_keywords = intent_data['genre_keywords'][genre]
        if any(keyword in book_desc or keyword in book_title for keyword in genre_keywords):
            score += 30
            reasons.append(f"matches {genre} genre")
            break
    
    if 'high_rating' in intent_data['quality_preferences'] and rating >= 4.0:
        score += 20
        reasons.append("high rating")
    
    if 'popular' in intent_data['quality_preferences'] and len(book_desc) > 200:
        score += 15
        reasons.append("comprehensive description suggests popularity")
    
    if intent_data['author_focused'] and author_name:
        score += 10
        reasons.append("has clear author attribution")
    
    stop_words = ['a', 'an', 'the', 'is', 'are', 'what', 'can', 'you', 'me', 'i', 'book', 'books']
    question_words = [word for word in intent_data['question_lower'].split() if word not in stop_words]
    
    desc_matches = sum(1 for word in question_words if word in book_desc)
    if desc_matches > 0:
        score += desc_matches * 5
        reasons.append(f"description matches {desc_matches} key terms")
    
    return {
        'book': book,
        'score': score,
        'reasons': reasons,
        'rating': rating
    }

In [13]:
def generate_ground_truth_response(top_books, intent_data):
    if not top_books:
        return "No relevant books found."
    
    top_book = top_books[0]
    book = top_book['book']
    title = book['book_title']
    author = book['author_name']
    rating = top_book['rating']
    
    detected_genres = intent_data['genres']
    quality_preferences = intent_data['quality_preferences']
    author_focused = intent_data['author_focused']
    
    if detected_genres:
        genre = detected_genres[0]
        if 'high_rating' in quality_preferences or rating >= 4.0:
            response = f"For a highly-rated {genre} book, I recommend '{title}' by {author} (rating: {rating:.1f})."
        else:
            response = f"A good {genre} book from the available options is '{title}' by {author}."
    
    elif 'high_rating' in quality_preferences:
        response = f"Among the highest-rated books available, '{title}' by {author} stands out with a {rating:.1f} rating."
    
    elif 'popular' in quality_preferences:
        response = f"'{title}' by {author} appears to be a popular choice based on the comprehensive information available."
    
    elif author_focused:
        response = f"I recommend '{title}' by the author {author}."
    
    else:
        if rating >= 4.0:
            response = f"I recommend '{title}' by {author}, which has a strong rating of {rating:.1f}."
        else:
            response = f"Based on the available books, '{title}' by {author} would be a good choice."
    
    if len(top_books) > 1 and top_books[1]['score'] > top_books[0]['score'] * 0.8:
        second_book = top_books[1]['book']
        response += f" You might also consider '{second_book['book_title']}' by {second_book['author_name']}."
    
    return response

In [14]:
def create_ground_truth(question, books_info):
    if not books_info:
        return "No relevant books found."
    
    intent_data = analyze_question_intent(question)
    
    book_scores = []
    for book in books_info:
        scored_book = calculate_book_score(book, intent_data)
        book_scores.append(scored_book)
    
    book_scores.sort(key=lambda x: x['score'], reverse=True)
    
    return generate_ground_truth_response(book_scores, intent_data)

In [15]:
def run_ragas_demo():
    print("🚀 Demo:\n")

    demo_questions = [
        "What's a good science fiction book with high ratings?",
        "Can you suggest a fantasy book by a popular author?", 
        "What's a highly rated mystery novel?",
        "Recommend a book with good reviews"
    ]

    questions, contexts_list, answers, ground_truths = [], [], [], []

    for i, question in enumerate(demo_questions, 1):
        print(f"\n📚 Question {i}: {question}")
        
        try:
            contexts, books_info = vector_search(question, top_k=3)
            if not contexts:
                print(f"No contexts found for question {i}")
                continue
                
            answer = generate_answer(question, contexts)
            print(f"Answer: {answer[:100]}...")
            ground_truth = create_ground_truth(question, books_info)
            print(f"Ground Truth: {ground_truth}")
            questions.append(question)
            contexts_list.append(contexts)
            answers.append(answer)
            ground_truths.append(ground_truth)
            
        except Exception as e:
            print(f"Error processing question {i}: {e}")
            continue

    if not questions:
        print("\nNo valid Q&A pairs generated.")
        return None

    eval_dataset = Dataset.from_dict({
        "question": questions,
        "contexts": contexts_list,
        "answer": answers,
        "ground_truth": ground_truths,
    })

    print("\n✨ Running Ragas evaluation...")
    try:
        result = evaluate(
            dataset=eval_dataset,
            metrics=[context_precision, faithfulness, context_recall],
            llm=chat_llm,
            embeddings=None
        )
        
        df = result.to_pandas()
        
        print("\n✨ Ragas Evaluation Results:")
        print(df)
        
        print("\✨ Averages:")
        
        for metric, value in df.mean(numeric_only=True).items():
            print(f"{metric}: {value:.3f}")
            
        df.to_csv("ragas_evaluation.csv", index=False)
        print("\nResults saved to 'ragas_evaluation.csv'")
        return result
        
    except Exception as e:
        print(f"Ragas evaluation failed: {e}")
        return None

In [16]:
try:
    results = run_ragas_demo()
    if results:
        print("\n🎉 Demo completed successfully!")
    else:
        print("\nDemo completed with issues.")
        
except Exception as e:
    print(f"\nError during demo: {e}")
    import traceback
    traceback.print_exc()

🚀 Demo:


📚 Question 1: What's a good science fiction book with high ratings?
Answer: "Light of the Jedi" by Charles Soule...
Ground Truth: For a highly-rated science fiction book, I recommend 'Light of the Jedi' by Charles Soule (rating: 4.2). You might also consider 'Legion' by Dan Abnett.

📚 Question 2: Can you suggest a fantasy book by a popular author?
Answer: Based on the context provided, I recommend "The Book of Time" by Guillaume Prévost....
Ground Truth: A good fantasy book from the available options is 'Our Pet' by S.M. Matthews. You might also consider 'The Book of Time' by Guillaume Prévost.

📚 Question 3: What's a highly rated mystery novel?
Answer: Human Nature by Jonathan Green...
Ground Truth: For a highly-rated mystery book, I recommend 'Human Nature' by Jonathan Green (rating: 3.6). You might also consider 'Lamb: The Gospel According to Biff, Christ's Childhood Pal' by Christopher Moore.

📚 Question 4: Recommend a book with good reviews
Answer: The Island of Doctor M

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]


✨ Ragas Evaluation Results:
                                          user_input  \
0  What's a good science fiction book with high r...   
1  Can you suggest a fantasy book by a popular au...   
2               What's a highly rated mystery novel?   
3                 Recommend a book with good reviews   

                                  retrieved_contexts  \
0  [Light of the Jedi by Charles Soule: Two hundr...   
1  [Alien Warrior's Mate by Vi Voxley: He's damn ...   
2  [The Island of Doctor Moreau by H.G. Wells: Ra...   
3  [The Island of Doctor Moreau by H.G. Wells: Ra...   

                                            response  \
0               "Light of the Jedi" by Charles Soule   
1  Based on the context provided, I recommend "Th...   
2                     Human Nature by Jonathan Green   
3          The Island of Doctor Moreau by H.G. Wells   

                                           reference  context_precision  \
0  For a highly-rated science fiction book, I rec... 